# RAG using Langchain

## Packages loading & import

In [ ]:
!pip install "langchain-core>=0.2.0,<0.3.0" \
             "langchain>=0.2.0,<0.3.0" \
             "langchain-community>=0.2.0,<0.3.0" \
             "langchain-huggingface>=0.0.3,<0.1.0" \
             "langchain-chroma>=0.1.0,<0.2.0" \
             "langchain-ollama>=0.1.0,<0.2.0" \
             "langchain-text-splitters>=0.2.0,<0.3.0" \
             "transformers>=4.39.0" \
             "accelerate>=0.28.0" \
             "sentence-transformers" \
             rank-bm25 \
             huggingface_hub \
             tqdm \
             beautifulsoup4

In [ ]:
import os
import json
import bs4
import nltk
import torch
import pickle
import numpy as np

# from pyserini.index import IndexWriter
# from pyserini.search import SimpleSearcher
from numpy.linalg import norm
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize

from langchain_community.llms import Ollama
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.embeddings import JinaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, TokenTextSplitter
from langchain.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from transformers import AutoModel, AutoModelForCausalLM, AutoTokenizer

from tqdm import tqdm

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')

## Hugging face login
- Please apply the model first: https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct
- If you haven't been granted access to this model, you can use other LLM model that doesn't have to apply.
- You must save the hf token otherwise you need to regenrate the token everytime.
- When using Ollama, no login is required to access and utilize the llama model.

In [ ]:
from huggingface_hub import login

hf_token = "Put your Hugging Face Access Token Key Here"
login(token=hf_token, add_to_git_credential=True)

In [ ]:
!huggingface-cli whoami

## TODO1: Set up the environment of Ollama

### Introduction to Ollama
- Ollama is a platform designed for running and managing large language models (LLMs) directly **on local devices**, providing a balance between performance, privacy, and control.
- There are also other tools support users to manage LLM on local devices and accelerate it like *vllm*, *Llamafile*, *GPT4ALL*...etc.

### Launch colabxterm

In [2]:
# TODO1-1: You should install colab-xterm and launch it.
# Write your commands here.

In [ ]:
# TODO1-2: You should install Ollama.
# You may need root privileges if you use a local machine instead of Colab.

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
%xterm

In [ ]:
# TODO1-3: Pull Llama3.2:1b via Ollama and start the Ollama service in the xterm
# Write your commands in the xterm

## Ollama testing
You can test your Ollama status with the following cells.

In [ ]:
# Setting up the model that this tutorial will use
MODEL = "llama3.2:1b" # https://ollama.com/library/llama3.2:3b
EMBED_MODEL = "jinaai/jina-embeddings-v2-base-en"

In [ ]:
# Initialize an instance of the Ollama model
llm = Ollama(model=MODEL)
# Invoke the model to generate responses
response = llm.invoke("What is the capital of Taiwan?")
print(response)

## Build a simple RAG system by using LangChain

### TODO2: Load the cat-facts dataset and prepare the retrieval database

In [ ]:
!wget https://huggingface.co/ngxson/demo_simple_rag_py/resolve/main/cat-facts.txt

In [4]:
# TODO2-1: Load the cat-facts dataset (as `refs`, which is a list of strings for all the cat facts)
# Write your code here

In [ ]:
from langchain_core.documents import Document
docs = [Document(page_content=doc, metadata={"id": i}) for i, doc in enumerate(refs)]

In [ ]:
# Create an embedding model
model_kwargs = {'trust_remote_code': True}
encode_kwargs = {'normalize_embeddings': False}
embeddings_model = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
# TODO2-2: Prepare the retrieval database
# You should create a Chroma vector store.
# search_type can be “similarity” (default), “mmr”, or “similarity_score_threshold”
vector_store = Chroma.from_documents(
    # Write your code here
)
retriever = vector_store.as_retriever(
    # Write your code here
)

### Prompt setting

In [ ]:
# TODO3: Set up the `system_prompt` and configure the prompt.
system_prompt = # Write your code here
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

- For the vectorspace, the common algorithm would be used like Faiss, Chroma...(https://python.langchain.com/docs/integrations/vectorstores/) to deal with the extreme huge database.

In [ ]:
# TODO4: Build and run the RAG system
# TODO4-1: Load the QA chain
# You should create a chain for passing a list of Documents to a model.
question_answer_chain = # Write your code here

# TODO4-2: Create retrieval chain
# You should create retrieval chain that retrieves documents and then passes them on.
chain = # Write your code here


In [ ]:
# Question (queries) and answer pairs
# Write your code here
# Please load the questions_answers.txt file and prepare the `queries` and `answers` lists.
queries = [
# Questions queries
]
answers = [
# Corresponding answers
]

In [ ]:
for i, query in tqdm(enumerate(queries), total=len(queries)):
    # TODO4-3: Run the RAG system
    response = # Write your code here
    # The following lines perform evaluations.
    # if the answer shows up in your response, the response is considered correct.
    # Compute recall@1, recall@5 and Accuracy.
    # Store the questions, ground-truths and answers in a json file.

# TODO5: Improve to let the LLM correctly answer the ten questions.